In [1]:
import os
import pandas as pd
from GraphBuild import * 
import matplotlib.pyplot as plt
import random
import pickle

In [2]:
# Initialize Circuit
FolderName = os.path.dirname(os.path.realpath("__file__"))
DSSfile = r""+ FolderName+ "\ieee34Mod1.dss"
Ckt_obj = CircuitSetup(DSSfile)  #creating a DSS object instance

In [3]:
#-- Equivalent Graph
G_original =  build_graph(Ckt_obj)
nx.readwrite.gml.write_gml(G_original,"34busEx.gml") #Graph undirected with edge features and node features which are constant
node_list=list(G_original.nodes())
edge_list=list(G_original.edges())

In [4]:
### Read Sensors
with open('bus_sensors.pkl', 'rb') as f1:
     bus_sensors = pickle.load(f1)

with open('branch_sensors.pkl', 'rb') as f2:
     branch_sensors = pickle.load(f2)

In [5]:
### Loadshapes Restructure
input_file = 'LoadShape1.xlsx'
sheet_name = 'LoadShape1'  # Change to the sheet you want to split
df = pd.read_excel(input_file, sheet_name=sheet_name)

points_per_day = 24  # Assuming 1-hour resolution (24 points = 24 hours)
# Convert to flat list
loadshape_values = df.iloc[:, 0].values.tolist()
# Check total number of complete days available
num_days = len(loadshape_values) // points_per_day
# Slice the loadshape into daily
LoadShapes = []
for i in range(num_days):
    start_idx = i * points_per_day
    end_idx = (i + 1) * points_per_day
    daily_shape = loadshape_values[start_idx:end_idx]
    LoadShapes.append(daily_shape)

In [6]:
Scenarios  = []
scid = 0
### Scenario generation
NSc = 1 # parameter indicating no.of scenarios for each

for idx in range(NSc):
    print(scid)
    loadshape_day = random.choice(LoadShapes)
    ldmult = random.choice(loadshape_day)
    Ckt_obj.dss.Solution.LoadMult(ldmult)
    Ckt_obj.dss.Solution.Solve()
    V_node_Sc = {}
    V_unknown = {}
    for bus in bus_sensors:
        V=Bus(Ckt_obj,bus).Vmag
        V_node_Sc[bus] = V  
            
    for bus in node_list:
        if bus not in bus_sensors:
            V=Bus(Ckt_obj,bus).Vmag
            V_unknown[bus] = V  
            
    flow_branch_Sc = {}  
    for (u,v) in branch_sensors:
        branch_label = G_original[u][v]['Label']
        branch_device = G_original[u][v]['Device']
        branch_elem = f"{branch_device}.{branch_label}"
        branch_pflow = Branch(Ckt_obj,branch_elem).flow
        flow_branch_Sc[(u,v)] = np.sum(branch_pflow)
        
    Scenarios.append({'Index':scid, 'Sensor BusVoltages':V_node_Sc, 'Sensor BranchFlows':flow_branch_Sc, 'Unknown BusVoltages':V_unknown})
    scid = scid + 1


0


In [18]:
Scenarios

[{'Index': 0,
  'Sensor BusVoltages': {'810': array([0.        , 1.03653855, 0.        ]),
   '808': array([1.02566842, 1.03665141, 1.03657561]),
   '816': array([1.0293372 , 1.03263755, 1.03004372]),
   '848': array([1.0290516 , 1.03919763, 1.03833457]),
   '826': array([0.        , 1.02589435, 0.        ]),
   '854': array([1.01001685, 1.01358723, 1.01184666]),
   '832': array([1.03221329, 1.04264703, 1.04116799]),
   '850': array([1.02961923, 1.03281153, 1.03022375]),
   '806': array([1.0471536 , 1.04829592, 1.04834799]),
   'sourcebus': array([1.0499979 , 1.04999876, 1.04999868]),
   '840': array([1.02833783, 1.03865588, 1.03767445]),
   '846': array([1.02900956, 1.03916103, 1.03828306]),
   '890': array([0.94086954, 0.95742814, 0.95013885]),
   '862': array([1.02835584, 1.03866697, 1.03769807]),
   '860': array([1.0285136 , 1.03896302, 1.03780189])},
  'Sensor BranchFlows': {('860', '836'): 106.47070670905997,
   ('814', '814r'): 424.97082933788437,
   ('812', '814'): 1414.0648313